In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# The goal is to show that Vespa does `.fill()` when there is no need for that.

In [92]:
!(cd ../examples/ignore-fill-bundle && mvn -f pom.xml package)

[INFO] Scanning for projects...
[INFO] 
[INFO] ----------------< lt.jocas.examples:ignore-fill-bundle >----------------
[INFO] Building ignore-fill-bundle 0.0.1
[INFO]   from pom.xml
[INFO] --------------------------[ container-plugin ]--------------------------
[WARNING] Ignoring incompatible plugin version 4.0.0-beta-1: The plugin org.apache.maven.plugins:maven-resources-plugin:4.0.0-beta-1 requires Maven version 4.0.0-beta-3
[INFO] Latest version of plugin org.apache.maven.plugins:maven-resources-plugin failed compatibility check
[INFO] Looking for compatible RELEASE version of plugin org.apache.maven.plugins:maven-resources-plugin
[INFO] Selected plugin org.apache.maven.plugins:maven-resources-plugin:3.3.1
[WARNING] Ignoring incompatible plugin version 4.0.0-beta-2: The plugin org.apache.maven.plugins:maven-install-plugin:4.0.0-beta-2 requires Maven version 4.0.0-rc-2
[INFO] Latest version of plugin org.apache.maven.plugins:maven-install-plugin failed compatibility check
[INFO] Loo

In [8]:
# How with pyvespa to add jar into components dir?

In [35]:
# Create a sample VAP
import mycode.vap as vap
from vespa.package import (ApplicationPackage, Field, Schema, Document, RankProfile, Function, FirstPhaseRanking)

doc_schema = Schema(
    name="doc",
    document=Document(
        fields=[
            Field(
                name="id",
                type="int",
                indexing=["attribute"],
                attribute=["fast-search"],
            ),
        ]
    ),
    rank_profiles=[
        RankProfile(
            name='fields',
            inherits='unranked',
            first_phase="0",
            functions=[
                Function(
                    name='id',
                    expression='attribute(id)'
                )
            ],
            match_features=[
                'id'
            ],
        )
    ]
)

application_package = ApplicationPackage(
    name="test",
    schema=[doc_schema],
)

In [34]:
from vespa.deployment import VespaDocker

# In case running colima on macos run the following
# !sudo ln -sf $HOME/.colima/default/docker.sock /var/run/docker.sock
vespa_docker = VespaDocker(
    container_image="vespaengine/vespa:8.588.8",
)
app = vespa_docker.deploy(
    application_package=application_package,
)

Waiting for configuration server, 0/60 seconds...
Waiting for application to come up, 0/300 seconds.
Waiting for application to come up, 5/300 seconds.
Waiting for application to come up, 10/300 seconds.
Application is up!
Finished deployment.


In [24]:
client = vap.redeploy(vespa_docker, application_package)

In [81]:
# Create and feed 1 dummy doc
docs = [
    {
        'id': f'{1}',
        'fields': {
            'id': 1,
        }
    }
]

client.feed_iterable(docs, schema="doc", namespace="doc", callback=vap.callback)

In [37]:
client.query(body={
    'yql': 'select matchfeatures from sources * where true',
    'ranking': 'fields',
    'presentation.summary': 'default',
    'trace.level': 6,
}).json

{'trace': {'children': [{'message': "Using query profile 'default' of type 'root'"},
   {'message': 'Resolved properties:\n'},
   {'message': "Invoking chain 'vespa' [com.yahoo.prelude.statistics.StatisticsSearcher@native -> com.yahoo.prelude.querytransform.PhrasingSearcher@vespa -> ... -> federation@native]"},
   {'children': [{'timestamp': 0,
      'message': "Invoke searcher 'com.yahoo.prelude.statistics.StatisticsSearcher in native'"},
     {'timestamp': 14,
      'message': "Invoke searcher 'com.yahoo.prelude.querytransform.PhrasingSearcher in vespa'"},
     {'timestamp': 14,
      'message': "Invoke searcher 'com.yahoo.prelude.searcher.FieldCollapsingSearcher in vespa'"},
     {'timestamp': 14,
      'message': "Invoke searcher 'com.yahoo.search.yql.MinimalQueryInserter in vespa'"},
     {'message': 'YQL query parsed: [\nselect matchfeatures from sources * where true\nTRUE\n]'},
     {'timestamp': 17,
      'message': "Invoke searcher 'com.yahoo.search.yql.FieldFilter in vespa'"}

In [38]:
# ^ As expected: one call to Vespa to get the summary class, and on the second take fill is ignored.

In [41]:
from pathlib import Path

# Now let's prepare another application package with the new search chain
application_package.to_files(Path('_tmp'))
!mkdir _tmp/components/

!cp ../examples/ignore-fill-bundle/target/ignore-fill-bundle-0.0.1-deploy.jar _tmp/components/


mkdir: _tmp/components/: File exists


In [99]:
from vespa.package import ServicesConfiguration
from vespa.configuration.services import services, container, document_api, search, chain, searcher, \
    document_processing, content, documents, document, redundancy

services_config = ServicesConfiguration(
    application_name="test",
    services_config=services(
        container(id="test_container", version="1.0")(
            document_api(),
            document_processing(),
            search(
                chain(
                    searcher(
                        id="lt.jocas.examples.SkipFillSearcher",
                        bundle="ignore-fill-bundle",
                    ),
                    id="fields",
                    inherits='vespa'
                )
            )
        ),
        content(id="test_content", version="1.0")(
            redundancy(1),
            documents(
                document(type="doc", mode="index"),
            )
        )
    )
)

application_package = ApplicationPackage(
    name='test',
    schema=[doc_schema],
    services_config=services_config
)

tmp_vap_folder = '_tmp'
application_package.to_files(Path(tmp_vap_folder))

!mkdir _tmp/components/
!(cd ../examples/ignore-fill-bundle && mvn -f pom.xml package)
!cp ../examples/ignore-fill-bundle/target/ignore-fill-bundle-0.0.1-deploy.jar _tmp/components/
! vespa deploy {tmp_vap_folder} -t http://localhost:19071

mkdir: _tmp/components/: File exists
[INFO] Scanning for projects...
[INFO] 
[INFO] ----------------< lt.jocas.examples:ignore-fill-bundle >----------------
[INFO] Building ignore-fill-bundle 0.0.1
[INFO]   from pom.xml
[INFO] --------------------------[ container-plugin ]--------------------------
[WARNING] Ignoring incompatible plugin version 4.0.0-beta-1: The plugin org.apache.maven.plugins:maven-resources-plugin:4.0.0-beta-1 requires Maven version 4.0.0-beta-3
[INFO] Latest version of plugin org.apache.maven.plugins:maven-resources-plugin failed compatibility check
[INFO] Looking for compatible RELEASE version of plugin org.apache.maven.plugins:maven-resources-plugin
[INFO] Selected plugin org.apache.maven.plugins:maven-resources-plugin:3.3.1
[WARNING] Ignoring incompatible plugin version 4.0.0-beta-2: The plugin org.apache.maven.plugins:maven-install-plugin:4.0.0-beta-2 requires Maven version 4.0.0-rc-2
[INFO] Latest version of plugin org.apache.maven.plugins:maven-install-plugin 

In [101]:
client.query(body={
    'yql': 'select matchfeatures from sources * where true',
    'ranking': 'fields',
    'presentation.summary': 'default',
    'trace.level': 6,
    'searchChain': 'fields',
}).json

{'trace': {'children': [{'message': "Using query profile 'default' of type 'root'"},
   {'message': 'Resolved properties:\n'},
   {'message': "Invoking chain 'fields' [lt.jocas.examples.SkipFillSearcher@fields -> com.yahoo.prelude.statistics.StatisticsSearcher@native -> ... -> federation@native]"},
   {'children': [{'timestamp': 0,
      'message': "Invoke searcher 'lt.jocas.examples.SkipFillSearcher in fields'"},
     {'timestamp': 0,
      'message': "Invoke searcher 'com.yahoo.prelude.statistics.StatisticsSearcher in native'"},
     {'timestamp': 0,
      'message': "Invoke searcher 'com.yahoo.prelude.querytransform.PhrasingSearcher in vespa'"},
     {'timestamp': 0,
      'message': "Invoke searcher 'com.yahoo.prelude.searcher.FieldCollapsingSearcher in vespa'"},
     {'timestamp': 0,
      'message': "Invoke searcher 'com.yahoo.search.yql.MinimalQueryInserter in vespa'"},
     {'message': 'YQL query parsed: [\nselect matchfeatures from sources * where true\nTRUE\n]'},
     {'times